In [1]:
# only for macbook with AMD GPU
import os
import plaidml.keras
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"


In [ ]:
! gdown --id 1GFm7tkn1N6Yx9kbm3UXIDXjOpAL19_HF

Downloading...
From: https://drive.google.com/uc?id=1GFm7tkn1N6Yx9kbm3UXIDXjOpAL19_HF
To: /content/image dataset ecg.rar
46.8MB [00:01, 40.1MB/s]


In [ ]:
!pwd


/content


In [ ]:
!unrar x "/content/image dataset ecg.rar"

Streaming output truncated to the last 5000 lines.
Extracting  test_imgs/0/4347.png                                          84%  OK 
Extracting  test_imgs/0/435.png                                           84%  OK 
Extracting  test_imgs/0/4351.png                                          84%  OK 
Extracting  test_imgs/0/4352.png                                          84%  OK 
Extracting  test_imgs/0/4353.png                                          84%  OK 
Extracting  test_imgs/0/4359.png                                          84%  OK 
Extracting  test_imgs/0/4363.png                                          84%  OK 
Extracting  test_imgs/0/4364.png                                          84%  OK 
Extracting  test_imgs/0/4367.png                                          84%  OK 
Extracting  test_imgs/0/437.png                                           84%  OK 
Extracting  test_imgs/0/4371.

In [2]:
import pandas as pd, matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import cv2
from tqdm import tqdm_notebook as tqdm
import gc

In [3]:
train_dir = './filter/train_imgs_128'
valid_dir = './filter/valid_imgs_128'
test_dir = './filter/test_imgs_128'

In [4]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
import cv2
import os
from tqdm.notebook import tqdm
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten , ELU
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, MaxPool2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import keras as keras
import keras.backend as K
from keras.models import load_model

In [5]:
# # if no split
train_datagen = ImageDataGenerator(rescale=1./255.0)


valid_datagen = ImageDataGenerator(rescale=1./255.0)
test_datagen = ImageDataGenerator(rescale=1./255.0)


In [26]:
batch_size = 4

In [27]:
# if no split
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    subset='training',class_mode='categorical', color_mode ='grayscale') # set as training data

validation_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',color_mode ='grayscale') # set as validation data
trest_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',color_mode ='grayscale') # set as validation data

Found 16320 images belonging to 5 classes.
Found 5440 images belonging to 5 classes.
Found 5440 images belonging to 5 classes.


In [28]:
model = None

In [29]:
model = Sequential()

model.add(Conv2D(32, (3,3),strides = (1,1), input_shape = (128,128,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(Conv2D(64, (3,3),strides = (1,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2, 2), strides= (2,2)))

model.add(Conv2D(128, (3,3),strides = (1,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(Conv2D(128, (3,3),strides = (1,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2, 2), strides= (2,2)))

model.add(Conv2D(256, (3,3),strides = (1,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(Conv2D(256, (3,3),strides = (1,1),kernel_initializer='glorot_uniform'))

model.add(ELU())

model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2, 2), strides= (2,2)))

model.add(Flatten())

model.add(Dense(2048))

model.add(ELU())

model.add(BatchNormalization())

model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 126, 126, 32)      320       
_________________________________________________________________
elu_22 (ELU)                 (None, 126, 126, 32)      0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 126, 126, 32)      128       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 124, 124, 64)      18496     
_________________________________________________________________
elu_23 (ELU)                 (None, 124, 124, 64)      0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 124, 124, 64)      256       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 62, 62, 64)        0         
__________

In [31]:
import tensorflow as tf

In [32]:
history = model.fit_generator(
    train_generator,
    class_weight = [0.64700285, 0.89650626, 1.36580467, 0.80342638, 2.76142132],
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),],
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=50)

Epoch 1/50
4080/4080 [==============================] - 1758s 431ms/step - loss: nan - acc: 0.3113 - val_loss: nan - val_acc: 0.3075
Epoch 2/50
4080/4080 [==============================] - 1732s 424ms/step - loss: nan - acc: 0.3093 - val_loss: nan - val_acc: 0.3075
Epoch 3/50
4080/4080 [==============================] - 1727s 423ms/step - loss: nan - acc: 0.3093 - val_loss: nan - val_acc: 0.3075
Epoch 4/50
3246/4080 [======================>.......] - ETA: 5:47 - loss: nan - acc: 0.3097

KeyboardInterrupt: 

In [ ]:
model.save("CNN_10_E_A_97.h5")

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [9]:
from keras.models import load_model


In [10]:
!ls

BALANCED IMAGE WITH FILTER.zip
CNN_10_E_A_97.h5
CNN_Image.ipynb
CNN_Image_Train_Only.ipynb
Images 256 with filter.zip
all_in_one_MIT_BIH_DATASET_32_5_2021.csv
ecgScratchEpoch2.hdf5
test_imgs
test_imgs_256
train_imgs
train_imgs_256


In [11]:
model = load_model('ecgScratchEpoch2.hdf5')













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 64)      1792      
_________________________________________________________________
elu_1 (ELU)                  (None, 126, 126, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 126, 126, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
elu_2 (ELU)                  (None, 124, 124, 64)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 124, 124, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
__________